In [ ]:
%matplotlib inline

# Deep Learning: Neural Networks
Essentially a branch of AI that closely tries to mimic how the human brain works. 
- Just like humans learn from experience, a deep learning algorithm can perform a task repeatedly, each time tweaking it to improve the outcome.

- In traditional machine learning, the algorithm is given a set of relevant features to analyze. However, in deep learning, the algorithm is given raw data and decides for itself what features are relevant.

*Training these Neural Networks take a lot of computational power. Pytorch really helps in the computational aspect as well as simplicity in programming the processes!*

https://www.youtube.com/watch?v=bfmFfD2RIcg (0:43)
![picture](https://drive.google.com/uc?id=1oL0IRADhSzDcwGRxOMh4T_JpetT3oEt1)

## 4 Steps to training Deep Learning Models

1. Data
2. **Model**
3. objective function; loss function
4. optimizer

We have already seen how to process the data in previous session!
This section will focus on the implementation of the Model.


# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data.

The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. 

Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. 

Let's check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we continue to use the CPU.

recall: If you’re using Colab, allocate a GPU by going to `Runtime > Change runtime type > GPU`.



In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the Class
We define our neural network by subclassing ``nn.Module``.

- Initialize the neural network layers in ``__init__``. 

- Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method. 

- Using a class to define our neural network makes the code very clean, and easily alterable.

We will break this down further below!


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print its structure.

By moving it to the GPU device, it allows to perform faster computations due to the GPU's ability to compute matrix operations quickly.



In [6]:
model = NeuralNetwork().to(device)
print(model) # For sanity! always check to see if this is what your structure intended!

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
- Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.

Softmax or also known as the "squashing" function makes any range of values sum to 1, or in other words converts it into a probability distribution!
- The network does not output a probability but rather scalar values called logits.



In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # an effect of inheriting nn.Module
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(pred_probab)
print(f"Predicted class: {y_pred}")

tensor([[0.1024, 0.0992, 0.1028, 0.0882, 0.0903, 0.1076, 0.1050, 0.1076, 0.0939,
         0.1030]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([7], device='cuda:0')


Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).

**Why do we flatten?**

Because we are using a Fully connected neural network layers! If we use a convolutional neural network, the height and width of the network should be maintained.


In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.




In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2975, -0.4780, -0.5673,  0.1773, -0.0771,  0.0989,  0.1470, -0.1311,
          0.4330, -0.4072,  0.1351, -0.1589, -0.7450, -0.1268, -0.6485,  0.2235,
         -0.6105,  0.2965,  0.0558, -0.4895],
        [ 0.5204, -0.4248, -0.5295,  0.4683, -0.1476,  0.2489,  0.4604, -0.3918,
          0.1053, -0.7214, -0.1279, -0.0559, -0.5307,  0.3835, -0.4957,  0.0658,
         -0.3237, -0.1884,  0.2070, -0.3767],
        [ 0.4987, -0.0744, -0.2513,  0.2511, -0.2907, -0.0714,  0.5542, -0.3855,
          0.3734, -0.6238,  0.0904, -0.5368, -0.6176, -0.1461, -0.1862, -0.2268,
         -0.1915,  0.0299, -0.1312, -0.1932]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2975, 0.0000, 0.0000, 0.1773, 0.0000, 0.0989, 0.1470, 0.0000, 0.4330,
         0.0000, 0.1351, 0.0000, 0.0000, 0.0000, 0.0000, 0.2235, 0.0000, 0.2965,
         0.0558, 0.0000],
        [0.5204, 0.0000, 0.0000, 0.4683, 0.0000, 0.2489, 0.4604, 0.0000, 0.1053,
         0.0000, 0.0000, 0.0000, 0.0000, 0.3835, 0.00

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

###  nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in $[-\infty, \infty]$ - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.
- Can be useful for debugging




In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 3.4440e-02, -4.7709e-05, -3.4345e-03,  ..., -1.9932e-02,
         -2.4304e-02, -3.3485e-02],
        [-2.4204e-02,  7.9442e-03, -6.5458e-04,  ...,  6.5797e-03,
         -3.5401e-02,  6.4008e-03]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0225, -0.0005], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0276,  0.0367,  0.0152,  ...,  0.0101,  0.0441, -0.0352],
        [ 0.0335,  0.0244, -0.0211,  ..., -0.03

## Exercise

Let's try to create our own Neural Network!
- input will be $16 \times 16$,
- 2 layers with 
- 256 hidden units in first layer,
- 10 units in the output layer.



In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        ###############Let's fill in below#################
        # Hint: nn.Sequential and 2 nn.Linear, 1 ReLU()
        

        ###################################################

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



In [ ]:
input_image = torch.rand(10,16,16)

####### FILL IN BELOW ############
# Initialize the network!

# Print the network to see if this is what you wanted!

# inference the input_image through the network we created and defined above!!

# print the shape of the output also!!



In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        ###############Let's fill in below#################
        # Hint: nn.Sequential and 2 nn.Linear, 1 ReLU()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16*16, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )
        ###################################################

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



In [ ]:
input_image = torch.rand(10,16,16)

# Initialize the network!
model = NeuralNetwork()
# Print the network to see if this is what you wanted!
print(model)

# TODO inference the input_image through the network we created and defined above!!
out = model(input_image)
# print the shape of the output also!!

print(out.shape)

torch.Size([10, 10])


## Further Reading

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

